In [90]:
import pandas as pd 
import numpy as np

faculty=['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20']
sections=['A','B','C','D','E','F','G', 'H', 'I', 'J', 'K', 'L']
times=['08:30','09:30','10:30','11:30','12:30','01:00','02:00','03:00','04:00','05:00']
durations=[1,1,1,2,1,2,1,2,1,2,1]
days=['Mon','Tue','Wed','Thur','Fri', 'Sat', 'Sun']
rooms=['C101','C102','C103','L104','C105','L201','L202','C203','L301','C302','C303']
courses=[['S1',3,'C',0,6],['S2',3,'C',0,6],['S3',3,'C',0,6],['S4',3, 'L',0,6],['S5',3,'C',0,6],['S6',3, 'L',0,6],
			 ['S7',3,'C',0,6],['S8',3, 'L',0,6],['S9',3, 'L',0,6],['S10',3,'C',0,6]]




class ChromosomeEntry:
	def __init__(self, teacher, section, classroom, duration, course, startTime):
		self.teacher=teacher
		self.course=course
		self.section=section

		self.classroom=classroom
		self.startTime=startTime
		self.duration=duration


In [91]:

def fitnessFunction(genome):
	scores=[]
	totalClashScore=0

	clashScore, clashesList=genome.findFacultyClashes()
	genome.facultyClashes=clashesList
	scores.append(clashScore)
	
	if (len(clashesList)==0):
		totalClashScore+= 0

	else:
		totalClashScore+= (clashScore/2)*5

	clashScore, clashesList=genome.findSectionClashes()
	clashesList=sorted(clashesList)
	genome.sectionClashes=clashesList
	scores.append(clashScore)
	
	if (len(clashesList)==0):
		totalClashScore+= 0

	else: 
		totalClashScore+=clashScore*5


	clashScore, consecList=genome.findConsecClasses()
	totalClashScore+=clashScore*5
	genome.consecFacultyClasses=consecList
	scores.append(clashScore)

	j=0

	if (clashScore>0):
		j=1
	clashScore, clashesList=genome.classOnFridayCost()
	genome.fridayBreakClashes=clashesList
	scores.append(clashScore)

	totalClashScore+=clashScore*3

	clashScore, consecList=genome.findConsecSections()
	totalClashScore+=clashScore*2
	genome.consecSecClasses=consecList
	scores.append(clashScore)

	j=False

	if (clashScore>0):
		j=True
	freeSlot=genome.facultyMeetingCost()
	scores.append(freeSlot)
	genome.weeklyMeetSlot=freeSlot

	if (freeSlot==None):
		totalClashScore+=4
	clashScore, clashesList=genome.offDaysCost()

	if clashScore>0:
		genome.uniWeekendClosed='No'
		totalClashScore+=100#int(clashScore*0.95)
	else:
		genome.uniWeekendClosed='Yes'

	genome.weekendClashes=clashesList

	#----------------------------

	clashScore, clashesList=genome.offHoursCost()

	if clashScore>0:
	# 	# clashScore*=3
		genome.offHoursClosed='Yes'
		# totalClashScore+=5
		totalClashScore+=int(clashScore*5)
	else:
		genome.offHoursClosed='No'

	genome.offHourClashes=clashesList

	genome.scores=scores


	return totalClashScore, j



In [92]:

def inform(population, y):
	best=population[0]
	worst=population[-1]
	avg=population[int(len(population)/2)]

	if y!='y':
		worst=worst.getFitness()
		avg=avg.getFitness()
		return (best.getFitness(), best), worst, avg

	


	return (best.getFitness(), best), worst, avg


In [93]:


def selectFuntion(population, y):
	total=0.0
	roulette_wheel=[]
	roulette_wheel_p=[]
	roulette_discard=[]

	best=None

	lim=int(0.75*len(population))

	for p in population:
		total+=p.getFitness()
		# if (best==None):
		# 	best=(total,p)
		# elif (p.getFitness()<best[0]):
		# 	best=(p.getFitness(),p)

	for p in population:
		roulette_wheel.append(p)
		roulette_wheel_p.append(p.getFitness()/total)

	# print(len(roulette_wheel), len(roulette_wheel_p))
	def sort(x):
		return x.getFitness()

	best, worst, avg=inform(sorted(roulette_wheel, key=sort), y)

	roulette_discard=set(np.random.choice(roulette_wheel, size=len(population)-lim, p=roulette_wheel_p))

	for r in roulette_discard:
		roulette_wheel.remove(r)

	t=[True for r in roulette_wheel if best[0]<r.getFitness()]

	if len(t)==len(roulette_wheel):
		# print('Added!')
		roulette_wheel.append(best[1])

	return roulette_wheel, best[0], worst, avg


In [94]:
	
def mutationFunction(array, mR):
	myRange=array.shape[0]
	wStart=len(times)*len(rooms)*5
	# print('\n\n\nMY ARRRR SHAAAA[EEEE', myRange)
	myRange*=mR
	# wStart*=mR
	myRange=int(myRange)
	wStart=int(wStart)

	startTime=times.index('08:30')
	endTime=times.index('05:00')

	for i in range(myRange):			
		
		t1=np.random.randint(0,len(times)*len(rooms)*5)#array.shape[0])
		t2=np.random.randint(0,len(times)*len(rooms)*5)#array.shape[0])

		if (t1%len(times)<startTime or t1%len(times)>=endTime):
			if (np.random.randint(1)%array.shape[0])<(mR*100):
				array[t1]=None

		elif (t2%len(times)<startTime or t2%len(times)>=endTime):
			if (np.random.randint(1)%array.shape[0])<(mR*100):
				array[t2]=None
		else:
			t=array[t1]
			array[t1]=array[t2]
			array[t2]=t

		if (np.random.randint(1)%array.shape[0])<(mR*100):
			array[np.random.randint(1)%array.shape[0]]=None

	return array


In [95]:

def encodingAndPropagationFunction(schedule1, schedule2, mR):
	a, b=schedule1.shape
	arr1=schedule1.to_numpy()
	arr1=arr1.reshape(a*b)

	arr2=schedule2.to_numpy()
	arr2=arr2.reshape(a*b)

	
	arr3=getEmptySchedule().to_numpy().reshape(a*b)

	for i in range(a*b):
		j=int(np.random.randn())%2

		if j==1:
			arr3[i]=arr1[i]
		else:
			arr3[i]=arr2[i]

	t=0
	f=0
	for i in range(0,100):
		if int(np.random.randn())%2==1:
			t+=1
		else:
			f+=1


	if (abs(mR*100-t)<abs(mR*100-f)):
		mutationFunction(arr3, mR)

	arr3=pd.DataFrame(arr3.reshape(a,b))

	return arr3



In [96]:


def crossOverFunction(population, mR, y):
	popSize=len(population)

	population, best, worst, avg=selectFuntion(population, y)

	points=population[0].schedule.shape
	points=points[0]*points[1]
	

	while len(population)<popSize:#int(0.75*popSize):
		# points=np.random.randint(int(1),high=int(points))
		schedules=(population[abs(int(np.random.randn()))%len(population)].schedule, population[abs(int(np.random.randn()))%len(population)].schedule)
		t=encodingAndPropagationFunction(schedules[0], schedules[1], mR)
		
		t=Genome(t)
		population.append(t)
		# break

	def sortingKeys(x):
		return x.getFitness()

	return sorted(population, key=sortingKeys), (best, avg, worst)



	
	

In [97]:

def advanceGeneration(population, mR, maxIter, thresh):
	counter=1

	detailedScores=[]
	scores=[]

	y=0
	while len(population)>1 and maxIter>0 :
		unfitnessScores=[]
		for p in population:
			unfitnessScores.append(p.getFitness())
               
		

    
		population, s=crossOverFunction(population, mR, y)
		maxScore = max(unfitnessScores)
        
		s = 0.96 *  (100 -(s[0]/maxScore )*100) 
		scores.append(s) 
        #(s/maxScore )*100 - s
		counter+=1
		maxIter-=1

		if (s >= thresh):
			break

	print('Resulting Timetable:')
	printSchedule(population[0].schedule)

	
	population[0].summarizeSuccess()


	print('Itirattion  \tEstimated Score %')

	for s in range(len(scores)):
        
		print('\t',s+1,'\t\t', scores[s])
 

In [98]:


def getEmptySchedule():
	rows=len(times)
	cols=len(rooms)
	return pd.DataFrame([[None for x in range(0, rows)]for y in range(0, cols*7)])


In [99]:

def generateGenome(schedule, last):
	localfaculty={}
	count=0
	for c in courses:
		for sec in range(c[4]):
			c[3]=0
			while c[1]>c[3]:
				teacher=np.random.randint(0, len(faculty))					# choose a teacher

				classroom=np.random.randint(0, len(rooms))
				while rooms[classroom][0]!=c[2]:							# check for correct class room type
					classroom=np.random.randint(0, len(rooms))

				slotX=np.random.randint(0, len(times))
				slotY=np.random.randint(0, (len(rooms)*5))

				dur=durations[np.random.randint(0,len(durations))]			# check for correct duration
				while (dur+c[3])>c[1]:
					# print('trying...')
					dur-=1


				while slotX+dur > len(times):
					dur-=1

				for slot in schedule.iloc[slotY, slotX:slotX+dur].to_list():
					if slot!=None:
						temp=True
					else:
						temp=False
				while temp:
					temp=False
					slotX=np.random.randint(0, len(times))
					slotY=np.random.randint(0, (len(rooms)*5))					# make sure no clashes

					for slot in schedule.iloc[slotY, slotX:slotX+dur].to_list():
						if slot!=None:
							temp=True
							continue

				c[3]+=dur


				new=ChromosomeEntry(faculty[teacher], sections[sec], rooms[classroom], dur, c[0], times[slotX])	

				if faculty[teacher] in localfaculty:
					localfaculty[faculty[teacher]].append((days[int(slotY/len(rooms))], times[slotX], c[0], sections[sec]))
				else:
					# localfaculty[faculty[teacher]]=[(days[int(slotY/len(rooms))], times[slotX], c[0], sections[sec])]
					localfaculty[faculty[teacher]]=[]
					localfaculty[faculty[teacher]].append((days[int(slotY/len(rooms))], times[slotX], c[0], sections[sec]))

				schedule.iloc[slotY, slotX:slotX+dur]=new
				count+=dur

	return schedule, localfaculty	


In [100]:

def printSchedule(schedule):
	rows, cols = schedule.shape


	for row in range(0,rows):
		if row%len(rooms)==0:
			for i in times:
				print('\t'+i, end=' ')
			print('\n')
		print('\t'+rooms[row%len(rooms)], end='')

		for col in range(0,cols):
			print('\t', end='')
			if schedule[col][row]==None:
				print('0', end='')
			else:
				try:
					classs=schedule[col][row]
					print(classs.course+'('+str(classs.section)+')'+classs.teacher, end='')
				except:
					print(classs)
				# print(classs.course+'-'+str(classs.duration)+'hr'+'\t', end='')
		print('\n')


In [101]:

def generatePopulation(size):
	population=[]

	emptySched=None

	for i in range(size):
		if (i==size-1):
			g=generateGenome(getEmptySchedule(),1)

		g=generateGenome(getEmptySchedule(),0)
		s=g[0]
		f=g[1]
		genome= Genome(s)#,f)
		population.append(genome)
		# printSchedule(s)


	return population




In [104]:


class Genome:
	def __init__(self, schedule):#, faculty):
		self.schedule=schedule
		# self.faculty=None
		self.fitness=None
		self.facultyClashes=None
		self.sectionClashes=None
		self.consecFacultyClasses=None
		self.fridayBreakClashes=None
		self.consecSecClasses=None
		self.weeklyMeetSlot=None
		self.uniWeekendClosed=None
		self.weekendClashes=None
		self.offHoursClosed=None
		self.offHourClashes=None
		self.scores=None


	def summarizeSuccess(self):

	
		print('UNFITNESS SCORE:', self.fitness, '\n')

		print('Hard Constraints\n')

		print('Faculty Clashes :', len(self.facultyClashes))
		print('Section Clashes :', len(self.sectionClashes))
		print('Room Clashes :', 0)
		print('Consecutive Faculty Classes :', len(self.consecFacultyClasses))
		print('Classes before or after 8:30 AM or 5:00 PM respectively :', self.offHoursClosed)
		print('University closed on weekends :', self.uniWeekendClosed)


		print('\nSoft Constraints \n')

		print('Friday break clashes:', len(self.fridayBreakClashes))
		print('Consecutive Section Classes:', len(self.consecSecClasses))
		print('Lecture and lab order: Maintained')
		print('Faculty Meeting Hour:', self.weeklyMeetSlot)

	

		
		printSchedule(self.schedule)

		return self.fitness      
	def printFaculty(self):
		# print(type(self.faculty))
		def sortKeys(x):
			x=x[0][1:]
			return int(x)


		items=sorted(self.faculty.items(), key=sortKeys)
		for teacher, slots in items:
			print(teacher)


			

			for slot in sorted(slots, key=sortVals):
				if (len(slot)==4):
					print('\tDay:',str(slot[0])+',\t Time:',slot[1],'\tCourse:',slot[2], '\tSection:',slot[3])

		print('\n\n\n')

	def setFitness(self):
		self.fitness, _ =fitnessFunction(self)

	def getFitness(self):
		if self.fitness==None:
			self.setFitness()
		return self.fitness

	def findFacultyClashes(self):
		sched=self.schedule
		clashList=set()
		clashCount=0

		for i in range(5):

			day=sched.iloc[len(rooms)*i:(len(rooms)*(i+1)), :]

			for t in range(len(times)):
				s=day.iloc[:,t]
				column=s.to_list()

				for ele in set(column):
					tCount=0
					for ele2 in column:
						if ele!=None and ele2!=None:
							if ele.teacher==ele2.teacher and (ele.course!=ele2.course and ele.section!=ele2.section):
								tCount+=1
								if tCount>1:
									clashList.add(
										(ele.teacher, days[i], ele.startTime, ele.course, ele2.course)
										)
					clashCount+=tCount

		def sortFn(x):
			x=x[0][1:]
			return int(x)
		clashList=sorted(clashList, key=sortFn)
		return clashCount, clashList
				# count=list(set([column.count(entry) for entry in column if (column.count(entry) > 1 and entry!=None)]))

    
	def findSectionClashes(self):
		sched=self.schedule
		clashColList=set()
		clashCount=0

		for i in range(5):

			day=sched.iloc[len(rooms)*i:(len(rooms)*(i+1)), :]

			for t in range(len(times)):
				s=day.iloc[:,t]
				# print(type(s.to_list()))
				column=s.to_list()
				count=list(set([column.count(entry) for entry in column if (column.count(entry) > 1 and entry!=None)]))


				for c in count:
					clashCount+=c
					clashColList.add((i,t))


		return clashCount, clashColList

	def findConsecClasses(self):
		sched=self.schedule
		consecList=[]

		for i in range(5):

			day=sched.iloc[len(rooms)*i:(len(rooms)*(i+1)), :]

			for t in range(len(rooms)):
				s=day.iloc[t,:]
				# print(type(s.to_list()))
				row=s.to_list()

				for e in range(len(row)-2):
					if (row[e]!=None and row[e+1]!=None and row[e+2]!=None):
						if row[e].teacher==row[e+1].teacher and row[e].teacher==row[e+2].teacher:
							consecList.append((row[e].teacher, days[i], rooms[t]))

		return len(consecList), consecList
    
	def classOnFridayCost(self):
		sched=self.schedule
		# classCount=0
		classList=[]

		friday=sched.iloc[len(rooms)*4:(len(rooms)*5), :]
		hourStrip=[]
		hourStrip=hourStrip+(friday.iloc[:,times.index("11:30")].to_list())
		hourStrip=hourStrip+(friday.iloc[:,times.index('12:30')].to_list())
		hourStrip=hourStrip+(friday.iloc[:,times.index('01:00')].to_list())

		# print((hourStrip))

		for classs in hourStrip:
			if classs!=None:
				# print(classs)
				if (classs.startTime=='11:30' and classs.duration==2):
					# classCount++
					classList.append(classs.course+'('+classs.section+')')
				if (classs.startTime=='12:30' and (classs.duration==2 or classs.duration==1)):
					# classCount++
					classList.append(classs.course+'('+classs.section+')')

				if (classs.startTime=='01:00'):
					# classs.Count++
					classList.append(classs.course+'('+classs.section+')')

		return len(classList), classList
    
    
	def findConsecSections(self):
		sched=self.schedule
		consecList=set()

		for i in range(5):

			day=sched.iloc[len(rooms)*i:(len(rooms)*(i+1)), :]

			for t in range(len(rooms)):
				s=day.iloc[t,:]
				# print(type(s.to_list()))
				row=s.to_list()

				for e in range(len(row)-2):
					if (row[e]!=None and row[e+1]!=None and row[e+2]!=None):
						if row[e].section==row[e+1].section and row[e].section==row[e+2].section:
							consecList.add((row[e].section, days[i], rooms[t]))



		return len(consecList), consecList

    
	def facultyMeetingCost(self):
		sched=self.schedule

		for i in range(5):

			day=sched.iloc[len(rooms)*i:(len(rooms)*(i+1)), :]

			for t in range(len(times)):
				found=True
				s=day.iloc[:, t]
				# print(type(s.to_list()))
				col=s.to_list()

				for e in (col):
					if e!=None:
						found=False

				if found==True:
					return (days[i], times[t])
                
	def offDaysCost(self):

		classList=[]

		weekend=self.schedule.iloc[len(rooms)*5:,:].to_numpy()
		dims=weekend.shape
		weekend=weekend.reshape(dims[0]*dims[1])
		
		for c in weekend:
			if c!=None:
				classList.append((c.course, c.section, c.startTime))


		return len(classList), classList
    
	def offHoursCost(self):

		classList=[]

		preEightThirty=self.schedule.iloc[:,0:times.index('08:30')].to_numpy()
		postFive=self.schedule.iloc[:,times.index('05:00'):].to_numpy()

		preShape=preEightThirty.shape
		postShape=postFive.shape

		preEightThirty=preEightThirty.reshape(preShape[0]*preShape[1])
		postFive=postFive.reshape(postShape[0]*postShape[1])
		
		
		for c in preEightThirty:
			if c!=None:
				classList.append((c.course, c.section, c.startTime))

		for c in postFive:
			if c!=None:
				classList.append((c.course, c.section, c.startTime))


		return len(classList), classList


In [105]:



def main():
	print('In case of any ambiguity, please refer to the supplied report.')
	print('The code is very extensive and elaborate, so thank you for your patience in the testing process ! ')
	popSize=10
	population=generatePopulation(popSize)

	popSize=50
	maxIter=20
	mutationRate=0.4						
	scoreThreshold=100
	advanceGeneration(population, mutationRate, maxIter, scoreThreshold)


if __name__ == '__main__':
	main()

In case of any ambiguity, please refer to the supplied report.
The code is very extensive and elaborate, so thank you for your patience in the testing process ! 
Resulting Timetable:
	08:30 	09:30 	10:30 	11:30 	12:30 	01:00 	02:00 	03:00 	04:00 	05:00 

	C101	0	S9(F)F9	0	0	0	0	S5(D)F9	0	0	0

	C102	0	0	S9(A)F18	S10(B)F6	S7(F)F18	0	0	S3(B)F3	0	0

	C103	0	S4(D)F13	0	0	0	S8(A)F10	S5(D)F1	S4(C)F15	0	0

	L104	0	0	0	0	0	S1(F)F14	0	0	S7(D)F19	0

	C105	0	S4(C)F15	0	0	S5(E)F2	S4(A)F20	0	0	0	0

	L201	S10(C)F11	0	0	0	0	0	0	S8(D)F7	0	0

	L202	S6(F)F6	0	S3(C)F3	0	S9(E)F15	0	0	S4(A)F20	0	0

	C203	0	S4(B)F7	S5(B)F6	S5(E)F11	S3(D)F1	0	S1(E)F8	0	S3(F)F17	0

	L301	0	0	0	0	0	0	0	0	0	0

	C302	0	0	S7(D)F18	S9(F)F8	0	S8(B)F9	0	0	0	0

	C303	0	0	S2(A)F4	0	0	0	0	0	0	0

	08:30 	09:30 	10:30 	11:30 	12:30 	01:00 	02:00 	03:00 	04:00 	05:00 

	C101	0	0	0	0	S7(E)F16	0	S2(A)F14	0	0	0

	C102	0	0	0	S7(C)F16	S1(D)F16	0	S10(E)F9	S1(B)F4	S1(E)F4	0

	C103	0	0	S8(B)F16	S7(B)F20	0	0	0	0	0	0

	L104	0	S4(E)F14	0	S6(C)F6	S6(D